In [2]:
import pandas as pd
import os
pd.options.display.max_columns = 200

# Istruzioni

Per effettuare il controllo CUP è sufficiente salvare i dati nelle corrette cartelle e poi eseguire direttamente tutto il notebook. 

## Input

- **Output DIPE**: salvare i file di output forniti dal DIPE nella cartella '01_raw/CUP'
- **Dati Opencoesione**: salvare l'estrazione dei dati OpenCoesione, in formato .zip, nella cartella '01_raw' [(vedi qui - link)](https://opencoesione.gov.it/it/progetti/?q=&selected_facets=is_pubblicato:0&selected_facets=is_pubblicato:1&selected_facets=is_pubblicato:2)
- **Estrazione CUP**: salvare un'estrazione dei CUP da PAdigitale (dall'omonimo report Estrazione CUP) nella cartella '01_raw'

In [3]:
DIRNAME = '../data/01_raw/CUP'

dfs = []
lenghts = []
for filename in os.listdir(DIRNAME):
    if filename.endswith('.xlsx'):
        with pd.ExcelFile(os.path.join(DIRNAME, filename)) as fl:
            data = pd.read_excel(fl,sheet_name=0, header=0).copy()
            dfs.append(data)
            lenghts.append(len(data))
df = pd.concat(dfs, axis = 0, ignore_index=True).drop_duplicates()
df.to_csv('../data/02_primary/CUP_feedback.parquet', index=False, quoting=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16021 entries, 0 to 16020
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CODICE_CUP             16021 non-null  object        
 1   SOGGETTO_TITOLARE      16021 non-null  object        
 2   DATA_GENERAZIONE       16021 non-null  datetime64[ns]
 3   TEMPLATE               15618 non-null  float64       
 4   COPERTURA_FINANZIARIA  16021 non-null  object        
 5   LOCALIZZAZIONE         16021 non-null  object        
 6   COSTO                  16021 non-null  int64         
 7   FINANZIAMENTO          16021 non-null  int64         
 8   DESCRIZIONE_AGGREGATA  16021 non-null  object        
 9   NATURA                 16021 non-null  object        
 10  TIPOLOGIA              16021 non-null  object        
 11  SETTORE                16021 non-null  object        
 12  SOTTOSETTORE           16021 non-null  object        
 13  C

In [3]:
FILENAME_OPENCOESIONE = 'progetti_esteso_20230430.zip'


df_OC = pd.read_csv(
    '../data/01_raw/'+FILENAME_OPENCOESIONE,
    compression='zip', 
    sep=';',
    quotechar='"', 
    low_memory=False, 
    usecols=['COD_LOCALE_PROGETTO','CUP','OC_TITOLO_PROGETTO','OC_DESCR_CICLO','OC_COD_CICLO','OC_TEMA_SINTETICO','FONDO_COMUNITARIO','OC_CODICE_PROGRAMMA','OC_DESCRIZIONE_PROGRAMMA','COD_STRUMENTO','DESCR_STRUMENTO','OC_STATO_FINANZIARIO','OC_STATO_PROGETTO','OC_STATO_PROCEDURALE'],
    header=0,
    #nrows=10
    )

In [6]:
df_OC.CUP.nunique()

1374455

In [17]:
FILE_PAD26 = 'Estrazione CUP-2023-07-17-16-57-41.xlsx'
FILE_TMP = 'Estrazione CUP-2023-07-19-09-01-50.xlsx'
with pd.ExcelFile(os.path.join('../data/01_raw', FILE_PAD26)) as fl:
    df_pad2026 = pd.read_excel(fl,sheet_name=0).copy() 


# COMPONENTE SPECIALE OCCASIONALE PER RIPORTARE MISURA E TIPOLOGIA ENTE
with pd.ExcelFile(os.path.join('../data/01_raw', FILE_TMP)) as fl:
    tmp_df = pd.read_excel(fl,sheet_name=0).copy() 
df_pad2026.columns = [col.upper().replace(' ','_') for col in df_pad2026.columns]
tmp_df.columns = [col.upper().replace(' ','_') for col in tmp_df.columns]
df_pad2026 = df_pad2026.merge(tmp_df.drop_duplicates(subset='CODICE_CUP'), how='left', on=['CODICE_CUP','NOME_DECRETO'])

df_pad2026.info()


/Users/rauldemaio/opt/anaconda3/envs/controllo_cup/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/rauldemaio/opt/anaconda3/envs/controllo_cup/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16048 entries, 0 to 16047
Data columns (total 4 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   CODICE_CUP                             16048 non-null  object
 1   NOME_DECRETO                           16048 non-null  object
 2   MISURA                                 16034 non-null  object
 3   APPLYING_ORGANIZATION:_TIPOLOGIA_ENTE  16034 non-null  object
dtypes: object(4)
memory usage: 626.9+ KB


In [18]:
assert df.CODICE_CUP.duplicated().sum()==0

tmp_df = df_pad2026.merge(df, on='CODICE_CUP', how='left').copy()
assert len(tmp_df)==len(df_pad2026)

In [19]:
# print(len(df_pad2026))
# print(tmp_df.info())
# print(tmp_df.STATO_PROGETTO.value_counts())
tmp_df.loc[tmp_df.STATO_PROGETTO.isna(),'STATO_PROGETTO'] = 'CUP INESISTENTE'

# Controlli da fare

Di seguito vengono fatti i seguenti controlli, in ordine di priorità:

1. CUP PNRR in OpenCoesione
2. CUP duplicato nel PNRR
3. CUP non esistente nei sistemi DIPE
4. CUP senza template
5. CUP template diverso


In [20]:
# concateno dataset da OpenCoesione (OC)
# df_OC = pd.concat([df_OC, df_OC2]).copy()

df_OC = df_OC.drop_duplicates(subset='CUP',keep='last').copy()

{'Decreto n.125 - 2 / 2022 - PNRR': 2203004.0,
 'Decreto n.125 - 3 / 2022 - PNRR': 2203004.0,
 'Decreto n.126 - 1 / 2022 - PNRR': 2205011.0,
 'Decreto n.126 - 2 / 2022 - PNRR': 2205011.0,
 'Decreto n.127 - 2 / 2022 - PNRR': 2203007.0,
 'Decreto n.127 - 3 / 2022 - PNRR': 2203007.0,
 'Decreto n.128 - 1 / 2022 - PNRR': 2205010.0,
 'Decreto n.128 - 2 / 2022 - PNRR': 2205010.0,
 'Decreto n.129 - 2 / 2022 - PNRR': 2203006.0,
 'Decreto n.129 - 3 / 2022 - PNRR': 2203006.0,
 'Decreto n.130 - 1 / 2022 - PNRR': 2205009.0,
 'Decreto n.130 - 2 / 2022 - PNRR': 2205009.0,
 'Decreto n.152 - 2 / 2022 - PNRR': 2210002.0,
 'Decreto n.152 - 3 / 2022 - PNRR': 2210002.0,
 'Decreto n.166 - 1 / 2022 - PNRR': 2204006.0,
 'Decreto n.166 - 2 / 2022 - PNRR': 2204006.0,
 'Decreto n.166 - 3 / 2022 - PNRR': 2204006.0,
 'Decreto n.167 - 1 / 2022 - PNRR': 2204005.0,
 'Decreto n.167 - 2 / 2022 - PNRR': 2204005.0,
 'Decreto n.175 - 1 / 2022 - PNRR': 2212001.0,
 'Decreto n.175 - 2 / 2022 - PNRR': 2212001.0,
 'Decreto n.1

In [28]:
# Primi check


## CUP double funding
DOUBLE_FUNDING_CUP = pd.merge( df_OC['CUP'], tmp_df['CODICE_CUP'], left_on ='CUP', right_on= 'CODICE_CUP',how='inner',validate="1:m").CUP.unique()
## CUP double project
DOUBLE_REQUEST_CUP = tmp_df.groupby(by = 'CODICE_CUP').filter(lambda x: len(x)>1).sort_values(by='CODICE_CUP').CODICE_CUP.unique()
## CUP non esistente nei sistemi DIPE
MISSING_CUP = tmp_df[tmp_df.STATO_PROGETTO=='CUP INESISTENTE'].CODICE_CUP.unique()
## Template mancante
MISSING_TEMPLATE = tmp_df[tmp_df.TEMPLATE.isna()].CODICE_CUP.unique()
#Template errato

map_sheet2template = tmp_df[['NOME_DECRETO','TEMPLATE']].groupby(['NOME_DECRETO']).TEMPLATE.agg(pd.Series.mode).to_dict()

# map_sheet2template['1.4.4 Altri Enti FINESTRA_76'] = 20220511
# map_sheet2template['1.4.3 Altri Enti App IOFINESTRA'] = 20220509
# map_sheet2template[ '1.4.3 pagoPA Altri EntiFINESTRA'] = 20220510

WRONG_TEMPLATE = [] 
WRONG_TEMPLATE = tmp_df[tmp_df.NOME_DECRETO.replace(map_sheet2template) != tmp_df.TEMPLATE].CODICE_CUP.unique()

In [30]:
def classify_CUP(CUP: str):
    '''
    Funzione per produrre una semplice classificazione dei CUP secondo una scala di priorità:
    1. CUP PNRR in OpenCoesione
    2. CUP duplicato nel PNRR
    3. CUP non esistente nei sistemi DIPE
    4. CUP senza template
    5. CUP template diverso
    '''

    if CUP in DOUBLE_FUNDING_CUP:
        return 'CUP già presente in OPEN COESIONE'
    elif CUP in MISSING_CUP:
        return 'CUP NON PRESENTE NEL FEEDBACK DIPE'
    elif CUP in DOUBLE_REQUEST_CUP:
        return "CUP PRESENTE IN PIU' CANDIDATURE"
    elif CUP in MISSING_TEMPLATE:
        return 'TEMPLATE MANCANTE'
    elif CUP in WRONG_TEMPLATE:
        return 'POSSIBILE TEMPLATE ERRATO'
    else:
        return 'NESSUN PROBLEMA RISCONTRATO AL MOMENTO'

In [32]:
# applica classificazione
tmp_df['CLASSIFICAZIONE_ISSUE_CUP']  = tmp_df.CODICE_CUP.apply(classify_CUP)

In [33]:
print(tmp_df.CODICE_CUP.count())
print(len(df_pad2026))
#crea aggregati
tmp_df.groupby(by=['CLASSIFICAZIONE_ISSUE_CUP']).CODICE_CUP.count()

16048
16048


CLASSIFICAZIONE_ISSUE_CUP
CUP NON PRESENTE NEL FEEDBACK DIPE           26
CUP PRESENTE IN PIU' CANDIDATURE              2
CUP già presente in OPEN COESIONE             1
NESSUN PROBLEMA RISCONTRATO AL MOMENTO    15316
POSSIBILE TEMPLATE ERRATO                   301
TEMPLATE MANCANTE                           402
Name: CODICE_CUP, dtype: int64

In [35]:
tmp_df.to_excel('../data/02_primary/CUP_ANALYSIS.xlsx', index=False)